# 0. 导入数据 & 数据预览

In [1]:
# 导入相关库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'pandas'

In [2]:
# 导入数据
data = pd.read_csv('train_split.csv')

# 获得数据属性名称
columns = data.columns.tolist()
print(data.shape)
print(columns)

# 查看数据前几项
data.head()

(23413, 28)
['ID', 'Customer_ID', 'Month', 'Name', 'Age', 'SSN', 'Occupation', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age', 'Payment_of_Min_Amount', 'Total_EMI_per_month', 'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance', 'Credit_Score']


,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x2184,CUS_0xab00,July,Markx,18,468-20-7719,Manager,31633.54,2930.128333,1,...,Good,846.45,37.934665,22 Years and 2 Months,No,50.768440,275.759794928842,Low_spent_Medium_value_payments,246.4845983624356,Poor
1,0x196a2,CUS_0x5fe0,May,Erwin Sebae,31,032-10-6627,Mechanic,33446.44,NaN,6,...,_,2953.68,37.895848,9 Years and 7 Months,Yes,133.355905,93.6504419059186,Low_spent_Large_value_payments,348.8139865924912,Poor
2,0x1e96c,CUS_0xc301,July,NaN,37,169-93-9953,Musician,21212.42,1910.701667,1,...,_,479.83,36.491037,23 Years and 1 Months,No,50.305036,172.47769262119868,Low_spent_Large_value_payments,238.28743816565944,Good
3,0x1b8a,CUS_0x1288,January,Taylorq,33,609-85-2725,Doctor,60938.13,5163.177500,10,...,Bad,3947.24,21.744884,4 Years and 10 Months,Yes,378.304673,166.48767607357289,High_spent_Small_value_payments,231.5254005497842,Poor
4,0x24b9d,CUS_0x899,April,Klaymanm,18,797-96-0406,Journalist,73057.16,5998.096667,4,...,Standard,2569.09,27.350833,9 Years and 6 Months,Yes,91.354188,66.23215374275686,High_spent_Medium_value_payments,692.2233254019017,Poor


In [3]:
data.describe()

,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Delay_from_due_date,Num_Credit_Inquiries,Credit_Utilization_Ratio,Total_EMI_per_month
count,19832.000000,23413.000000,23413.000000,23413.000000,23413.000000,22975.000000,23413.000000,23413.000000
mean,4141.187099,16.760347,22.776492,72.264298,22.534062,27.925223,32.215666,1342.878898
std,3252.627113,114.255920,131.197844,460.147341,16.543341,191.189327,5.121565,8027.044087
min,319.556250,-1.000000,0.000000,1.000000,-5.000000,0.000000,20.172942,0.000000
25%,1564.692500,3.000000,4.000000,7.000000,10.000000,3.000000,27.976271,33.767386
50%,3080.555000,6.000000,6.000000,15.000000,19.000000,6.000000,32.250408,73.981294
75%,5801.966667,8.000000,7.000000,24.000000,31.000000,10.000000,36.458782,169.936005
max,15204.633333,1794.000000,1498.000000,5788.000000,67.000000,2587.000000,49.522324,82236.000000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23413 entries, 0 to 23412
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        23413 non-null  object 
 1   Customer_ID               23413 non-null  object 
 2   Month                     23413 non-null  object 
 3   Name                      21019 non-null  object 
 4   Age                       23413 non-null  object 
 5   SSN                       23413 non-null  object 
 6   Occupation                23413 non-null  object 
 7   Annual_Income             23413 non-null  object 
 8   Monthly_Inhand_Salary     19832 non-null  float64
 9   Num_Bank_Accounts         23413 non-null  int64  
 10  Num_Credit_Card           23413 non-null  int64  
 11  Interest_Rate             23413 non-null  int64  
 12  Num_of_Loan               23413 non-null  object 
 13  Type_of_Loan              21019 non-null  object 
 14  Delay_